# Add Embeddings to Table

# ADD LINKS

# ADD DESCRIPTION

In [ ]:
PROJECT_NAME = "Bounding Box Classifier"
TRANSIENT_DATA_PATH = "../../transient_data"
BATCH_SIZE = 32
DATASET_NAME = "Balloons"

## Imports

In [ ]:
from copy import deepcopy

import numpy as np
import tlc
from tqdm.auto import tqdm

## Get Original Table

In [ ]:
# Open the Table created in the first notebook
original_table = tlc.Table.from_names(
    project_name=PROJECT_NAME,
    dataset_name=DATASET_NAME,
    table_name="original"
)

## Create New Table with Embeddings

In [ ]:
# Open the embeddings created in previous notebook
embeddings = np.load(TRANSIENT_DATA_PATH + "/bb_classifier_embeddings.npy")

# Create schema for embedding
embedding_schema = tlc.FloatVector3Schema()

# Create a schema for the new table
new_table_schema = deepcopy(original_table.rows_schema)
new_table_schema.values["bbs"].values["bb_list"].add_sub_schema("embedding", embedding_schema)

In [ ]:
# Create a TableWriter for the new table
table_writer = tlc.TableWriter(
    project_name=PROJECT_NAME,
    dataset_name=DATASET_NAME,
    table_name="embeddings_added",
    column_schemas=new_table_schema.values,
    if_exists="overwrite"
)

# Iterate over the rows of the original table and add the embeddings
i = 0
for row in tqdm(original_table):
    new_row = deepcopy(row)
    for bb in new_row["bbs"]["bb_list"]:
        bb["embedding"] = embeddings[i].tolist()
        i += 1

    table_writer.add_row(new_row)

# Create the new table
table_writer.finalize()
    